Dieses Notebook ist angelehnt an das *SciPy 2018 Scikit-learn Tutorial* von Andreas Mueller und Guillaume Lemaitre, verfügbar auf [GitHub](https://github.com/amueller/scipy-2018-sklearn).

# Praktikum: Session 4
In dieser Session soll ein Modell entwickelt werden, welches basierend auf der Passagierliste der HMS Titanic das Überleben der einzelnen Passagiere vorhersagt.

Dazu verwenden wir eine Version der Titanic-Daten von [hier](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls). Solch externe Daten können auf unterschiedliche Arten bereitgestellt werden:


*   direktes Einlesen aus der Quelle in einen Dateframe durch den Befehl `pd.read_excel('Adresse der Quelle')`.
*   Herunterladen der Quelle und einlesen vom gewählten Speicherort (ebefalls durch `pd.read_excel`). Wenn Sie Colab verwenden können Sie durch das Menü links den Dateibaum einblenden und dorthin Dateien hochladen. Den genauen Pfad der Datei (um ihn in den `pd.read_***` Befehl einzufügen, erhalten Sie am einfachsten durch einen Rechtsklick auf die Datei, Kontextmenü "Pfad kopieren".)

Pandas kann viele Datenformate nativ einlesen, so gibt es z.B. auch `pd.read_csv()` etc.

Diese Daten werden nun eingelesen. Dabei werden die Daten der ersten Zeile als Spaltennamen interpretiert. Diese lassen wir direkt anzeigen:

In [ ]:
import os
import pandas as pd

titanic = pd.read_csv('/content/titanic3.csv')
print(titanic.columns)


Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


Was hat es mit diesen Spalten auf sich?

Hier eine knappe Beschreibung:
```
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
survival        Survival
                (0 = No; 1 = Yes)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)
boat            Lifeboat
body            Body Identification Number
home.dest       Home/Destination
```

Welche dieser Features sollte man als kategorische Features auffassen, welche als numerische?

### Lösung

**Kategorische Features:** `name`, `sex`, `cabin`, `embarked`, `boat`, `body`, und `homedest`

**Weitere kategorische Features:** ``pclass`` - dieses ist zwar als Zahl angegeben, allerdings liegt hier keine algebraische Logik zugrunde.

### Auswahl der Daten
Durch das Einlesen der Daten oben haben wir einen Pandas DataFrame angelegt. Verschaffen Sie sich ein erstes Gefühl für die Daten, indem Sie die ersten fünf Einträge betrachten.

Ziel ist es, ein Modell zu bauen, welches für jeden Passagier vorhersagt, ob diese überlebt oder nicht. Welche Features sollte man für solch ein Modell sinnvollerweise verwenden? Was ist das gewünschte Label/Target?

- Legen Sie einen Vektor ``labels`` an, der die Labels enthält.
- Legen Sie entsprechend Ihrer Wahl eine Features-Matrix an.

Kontrollieren Sie, ob die erstellten Objekte sich so verhalten wie gewünscht, d.h. betrachten Sie diese kurz.

*Hinweise:*
- Welche sind für die Fragestellung offensichtlich irrelevant, welche enthalten die gesuchte Information bereits auf offensichtliche Weise?
- Einzelne Spalten können aus einem DataFrame ``df`` extrahiert werden durch ``df[['Spaltenname1', ..., 'SpaltennameN']]``
- Ist man nur an den Werten interessiert, d.h. an einem Vektor ohne die zugehörigen Indizierung, so kann man noch ``.values`` anhängen.

Der DataFrame ``features`` enthält nun nur noch nützliche Features, allerdings liegen diese noch nicht in einer nutzbaren Form vor (nicht-numerische Daten). Im Folgenden sollen die Daten aufbereitet werden.

## Datenaufbereitung mit Pandas
Da unsere Daten in einem DataFrame vorliegen, können wir Möglichkeiten der Bibliothek ``Pandas`` (in welcher DataFrames definiert sind) nutzen, um die nötigen Manipulationen durchzuführen.

Offensichtlich verändert werden müssen die Features ``sex`` und ``embarked``; die naheliegende Option ist ein One-Hot-Encoding. Eine solche One-Hot-Codierung aller nicht-numerischen Spalten kann in Pandas sehr einfach z.B. durch die Methode ``get_dummies()`` durchgeführt werden. Wenden Sie diese Methode auf die Daten an und betrachten Sie das Ergebnis.

*Hinweis:* Die Methode stammt aus der Bibliothek ``pandas``, welche wir ganz oben importiert haben; wir haben sie unter dem (Standard-)Kürzel ``pd`` bereitgestellt. Somit können Sie die gewünschete Methode aufrufen mit ``pd.get_dummies(zu_verarbeitende_Daten)`` aufrufen.

Oben hatten wir überlegt, dass es sinnvoll wäre, auch das Feature ``pclass`` als kategorisch aufzufassen. Verwenden Sie wieder die ``get_dummies()`` Methode, um auch ``pclass`` durch One-Hot-Encoding darzustellen.

*Hinweis:* ``get_dummies()`` kann das optionale Argument ``columns`` übergeben werden. Durch dieses kann eine Liste an Features explizit angegeben werden, die One-Hot kodiert werden sollen.

Damit liegen die Daten nun in einer geeigneten Form vor. Den DataFrame brauchen wir für unser Modell eigentlich nicht mehr, daher extrahieren wir die Daten daraus mit
    ```data = features_dummies.values```.

## Fehlende Daten
Wir müssen aber noch prüfen, ob evtl. Daten fehlen, d.h. ob manche Einträge den Wert ``NaN`` (not a number) haben und damit ungültig sind. Das kann z.B. mit Hilfe der Methode ``isnan()`` gemacht werden. Diese steht in der Bibliothek ``numpy`` bereit (welche standardmäßig mit dem ``np`` importiert wird) und liefert bei Anwendung auf ein Array ein Array der gleichen Form zurück, welches nur die Einträge ``True`` und ``False`` hat. ``True`` steht an jeder Position, an der das Ausgangsarray den Wert ``NaN`` hat, ``False`` überall dort, wo gültige Daten vorliegen.

Somit interessiert uns eigentlich nur, ob dieses große "``True/False``-Array" irgendwo den Wert ``True`` hat. Dies kann durch die Methode ``any()`` realisiert werden.

Da es also fehlende Daten gibt, müssen diese ersetzt werden. Dafür wollen wir den ``SimpleImputer`` verwenden.

**VORSICHT: Bevor ein Imputer verwendet wird, müssen die Daten in Trainings- und Testdaten aufgeteilt werden!** Warum?

- Teilen Sie die vorliegenden (und bereits aufbereiteten) Daten auf in Trainings- und Testdaten.
- Verwenden Sie den ``SimpleImputer`` um fehlende Werte zu ergänzen.
- Prüfen Sie nochmals, ob nun noch ``NaN`` vorhanden ist.

## Betrachtung von Modellen
Nun sind die Daten soweit aufbereitet, dass man ein Modell damit trainieren kann. Es stellt sich wie immer die Frage: Welches Ergebnis ist gut?

### Baseline: Dummy Classifier
Verwenden Sie den ``DummyClassifier`` mit der Strategie ``most_frequent``, um eine untere Grenze für eine akzeptable Vorhersagegenauigkeit zu erhalten. Was macht dieser Dummy Classifier?

*Hinweis:* Diesen Classifier finden Sie in ``sklearn.dummy``.

### Naive Bayes
Verwenden Sie einen Naive Bayes Classifier und bestimmen Sie dessen Genauigkeit.

### Ensemble Methode: RandomForestClassifier
Verwenden Sie den RandomForestClassifier und bestimmen Sie dessen Genauigkeit.
*Hinweis:* Diesen finden Sie in ``sklearn.ensemble``.

Entfernen Sie die Features ``embark`` und ``parch`` aus dem Datensatz und trainieren und bewerten Sie erneut das gleiche RandomForest Modell.

### Support Vector Classifier (SVC)
Verwenden Sie einen Support Vector Classifier und bestimmen Sie dessen Genauigkeit.

Der Classifier SVC biete im Gegensatz zu Naive Bayes einige Hyperparameter, die es zu wählen gilt. Verwenden Sie eine GridSearch, um eine geeignete Kombination zu finden.

Wir wollen das Grid um die besten gefundenen Hyperparameterwerte feiner machen in der Hoffnung, eine weitere Verbesserung zu erzielen: